In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import nltk
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [4]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_data.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
X = train_data['text']
y = train_data['target']

In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stop_words = stopwords.words('english')
stemmer = PorterStemmer()

X_prepro = []

for i in train_data['text']:
    i = i.split(" ")
    word = [stemmer.stem(j) for j in i if j not in stop_words]
    w = " ".join(word)
    X_prepro.append(w)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectores = CountVectorizer(max_features=50000,min_df=2)
X = vectores.fit_transform(X_prepro).toarray()

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
X_train[0]

array([0, 0, 0, ..., 0, 0, 0])

# Building Models

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold

SVC

In [12]:
from sklearn.svm import SVC

sv_model = SVC()
sv_model.fit(X_train,y_train)
sv_pred = sv_model.predict(X_test)
print(confusion_matrix(sv_pred,y_test))
print(classification_report(sv_pred,y_test))
print(accuracy_score(sv_pred,y_test))

[[819 227]
 [ 63 414]]
              precision    recall  f1-score   support

           0       0.93      0.78      0.85      1046
           1       0.65      0.87      0.74       477

    accuracy                           0.81      1523
   macro avg       0.79      0.83      0.80      1523
weighted avg       0.84      0.81      0.82      1523

0.8095863427445831


Randomforest classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
print(confusion_matrix(rf_pred,y_test))
print(classification_report(rf_pred,y_test))
print(accuracy_score(rf_pred,y_test))

[[806 246]
 [ 76 395]]
              precision    recall  f1-score   support

           0       0.91      0.77      0.83      1052
           1       0.62      0.84      0.71       471

    accuracy                           0.79      1523
   macro avg       0.77      0.80      0.77      1523
weighted avg       0.82      0.79      0.80      1523

0.788575180564675


In [14]:
# cv = RepeatedKFold(n_splits=10,n_repeats=3,random_state=1)
# rf_model = RandomForestClassifier()


# parameters = {'n_estimators': [15,25,30,35],
#               'criterion': ['gini','entropy'],
#                'max_features': [2,3,'auto'],
#                'max_depth': [2,4,6,None],
#                'min_samples_split': [2,5,7,10,12]}

# grid = GridSearchCV(rf_model,parameters,scoring='f1',cv=5)
# result = grid.fit(X_train,y_train)
# print("best_score:",result.best_score_)
# print("best_params:",result.best_params_)

XGBoost

In [15]:
from xgboost import XGBClassifier


xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)
xgb_pred = xgb_model.predict(X_test)
print(confusion_matrix(xgb_pred,y_test))
print(classification_report(xgb_pred,y_test))
print(accuracy_score(xgb_pred,y_test))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:09:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[792 231]
 [ 90 410]]
              precision    recall  f1-score   support

           0       0.90      0.77      0.83      1023
           1       0.64      0.82      0.72       500

    accuracy                           0.79      1523
   macro avg       0.77      0.80      0.78      1523
weighted avg       0.81      0.79      0.79      1523

0.7892317793827971


In [16]:
# xgb_model = XGBClassifier()
# parameters = {'max_depth':range(3,10,2),
#              'min_child_weight':range(1,6,2),
#              'gamma':[i/10.0 for i in range(0,5)],
#              'subsample':[i/10.0 for i in range(6,10)],
#              'colsample_bytree':[i/10.0 for i in range(6,10)],
#              'reg_alpha':[1e-5, 1e-2, 0.1, 1, 0.001, 0.005, 0.01, 0.05]}

# grid = GridSearchCV(xgb_model,parameters,scoring='f1',cv=5)
# result = grid.fit(X_train,y_train)
# print("best_score:",result.best_score_)
# print("best_params:",result.best_params_)

LightGBM

In [17]:
from lightgbm import LGBMClassifier

lgb_model = LGBMClassifier()
lgb_model.fit(X_train,y_train)
lgb_pred = lgb_model.predict(X_test)
print(confusion_matrix(lgb_pred,y_test))
print(classification_report(lgb_pred,y_test))
print(accuracy_score(lgb_pred,y_test))

[[781 220]
 [101 421]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83      1001
           1       0.66      0.81      0.72       522

    accuracy                           0.79      1523
   macro avg       0.77      0.79      0.78      1523
weighted avg       0.81      0.79      0.79      1523

0.7892317793827971


In [18]:
# lgb_model = LGBMClassifier()
# parameters = {
#     'bagging_fraction': (0.5, 0.8),
#         'bagging_frequency': (5, 8),
#         'feature_fraction': (0.5, 0.8),
#         'max_depth': (10, 13),
#         'min_data_in_leaf': (90, 120),
#         'num_leaves': (1200, 1550)
# }

# grid = GridSearchCV(lgb_model,parameters,scoring='f1',cv=5)
# result = grid.fit(X_train,y_train)
# print("best_score:",result.best_score_)
# print("best_params:",result.best_params_)

Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model.fit(X_train,y_train)
gnb_pred = gnb_model.predict(X_test)
print(confusion_matrix(gnb_pred,y_test))
print(classification_report(gnb_pred,y_test))
print(accuracy_score(gnb_pred,y_test))

[[420 130]
 [462 511]]
              precision    recall  f1-score   support

           0       0.48      0.76      0.59       550
           1       0.80      0.53      0.63       973

    accuracy                           0.61      1523
   macro avg       0.64      0.64      0.61      1523
weighted avg       0.68      0.61      0.62      1523

0.6112934996717005


In [20]:
from sklearn.naive_bayes import BernoulliNB

bnb_model = BernoulliNB()
bnb_model.fit(X_train,y_train)
bnb_pred = bnb_model.predict(X_test)
print(confusion_matrix(bnb_pred,y_test))
print(classification_report(bnb_pred,y_test))
print(accuracy_score(bnb_pred,y_test))

[[795 206]
 [ 87 435]]
              precision    recall  f1-score   support

           0       0.90      0.79      0.84      1001
           1       0.68      0.83      0.75       522

    accuracy                           0.81      1523
   macro avg       0.79      0.81      0.80      1523
weighted avg       0.83      0.81      0.81      1523

0.8076165462902167


In [21]:
from sklearn.naive_bayes import MultinomialNB

cnb_model = MultinomialNB()
cnb_model.fit(X_train,y_train)
cnb_pred = cnb_model.predict(X_test)
print(confusion_matrix(cnb_pred,y_test))
print(classification_report(cnb_pred,y_test))
print(accuracy_score(cnb_pred,y_test))

[[772 198]
 [110 443]]
              precision    recall  f1-score   support

           0       0.88      0.80      0.83       970
           1       0.69      0.80      0.74       553

    accuracy                           0.80      1523
   macro avg       0.78      0.80      0.79      1523
weighted avg       0.81      0.80      0.80      1523

0.7977675640183848


In [22]:
test_vectors = vectores.transform(test_data['text'])

In [23]:
my_submission = pd.DataFrame({'id': test_data.id, 'target': bnb_model.predict(test_vectors)})

In [24]:
my_submission.to_csv('submission.csv', index=False)

In [25]:
# sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
# sample_submission["target"] = bnb_model.predict(test_vectors)

In [26]:
# sample_submission.head()

In [27]:
# sample_submission.to_csv("submission.csv", index=False)